In [218]:
# Standards
import pandas as pd
import numpy as np

# API
import requests

# Automating
import time
import datetime
import warnings
import sys

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Set base url
url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='

In [38]:
# Set params
params = {
    'subreddit':'machinelearning',
    'size':100,
    'lang':True,
#     'before':1601384439
}

In [39]:
# Make request
res = requests.get(url,params)
res

<Response [200]>

In [40]:
df_ml = pd.DataFrame(res.json()['data'])

In [41]:
# Extract columns
df_ml = df_ml.loc[:,['title','author','created_utc','selftext','subreddit','id','media_only']]
df_ml

,title,author,created_utc,selftext,subreddit,id,media_only
0,[D] NLP and Sesame Street Papers,enclosed_mail,1601496204,I think subconsciously I like NLP because I ha...,MachineLearning,j2tk96,False
1,NLP and Sesame Street,enclosed_mail,1601495824,[removed],MachineLearning,j2tfxi,False
2,[P] Many PhD fellowships (ERC funded) in Data ...,tuyenttoslo,1601495770,"Dear all, \n\n\nThere will be two calls of P...",MachineLearning,j2tfcv,False
3,Fashion forecasting with machine learning [P],zero_redditer,1601495149,Hi everyone! This week I joined an exciting ch...,MachineLearning,j2t8gs,False
4,Fashion forecasting with machine learning,zero_redditer,1601495034,[removed],MachineLearning,j2t75k,False
...,...,...,...,...,...,...,...
95,[N] The $75000 COVID-19 Symptom Data Challenge...,SubstantialRange,1601400509,[https://www.symptomchallenge.org/](https://ww...,MachineLearning,j23wfb,False
96,"The $75000 COVID-19 Symptom Data Challenge, Fr...",SubstantialRange,1601400464,,MachineLearning,j23vv0,False
97,[D] Simplest way of doing distributed training...,datatuabun,1601400417,"Hi peeps, I was wondering if any of you have a...",MachineLearning,j23v9r,False
98,[R] Dendrite Net: A White-Box Module for Class...,MGangLiu,1601398890,Abstract \n\nThis paper presents a basic mach...,MachineLearning,j23bt7,False


In [11]:
df_wall = pd.DataFrame(res.json()['data'])

In [12]:
df_wall.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_text', 'author_flair_text_color',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
       'preview', 'pwls', 'removed_by_category', 'retrieved_on', 'score',
       'selftext', 'send_replies',

In [18]:
# Extract columns
df_wall = df_wall.loc[:,['title','author','created_utc','selftext','subreddit','id','media_only']]
df_wall

,title,author,created_utc,selftext,subreddit,id,media_only
0,Translation: “Huge fan of WSB. Will make sure ...,DeanYerdon,1601384371,,wallstreetbets,j1ypux,False
1,ONE OF US: From $330 million to $9 million - G...,Schrodingersdawg,1601383932,https://nypost.com/2020/09/28/allianz-accused-...,wallstreetbets,j1ylrv,False
2,Propane and Propane Related Products (SPH),Nolliethedog,1601383885,I'm betting a large increase in demand for pr...,wallstreetbets,j1ylb8,False
3,Update from mom who made 500+% on lithium stoc...,htotheizzostocks,1601383841,,wallstreetbets,j1ykvs,False
4,"congrats, retards",peepee_the_cat,1601383786,,wallstreetbets,j1ykew,False
...,...,...,...,...,...,...,...
95,Largest 4h Streak of 2020- Red is coming,TheRedditRef,1601356698,,wallstreetbets,j1t12v,False
96,Bull Szn,StonkCalls,1601356615,,wallstreetbets,j1t0g4,False
97,THIS is Value Investing,FloydMCD,1601356544,,wallstreetbets,j1szwz,False
98,“Got stuck in some waist deep at Alpental last...,Kommoncents1,1601356396,,wallstreetbets,j1sypp,False


In [51]:
def get_posts(subreddit, n_iter, epoch_right_now): # subreddit name and number of times function should run
    # store base url variable
    base_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='
    # instantiate empty list    
    df_list = []
    # save current epoch, used to iterate in reverse through time
    current_time = epoch_right_now
    # set up for loop
    for post in range(n_iter):
        # instantiate get request
        res = requests.get(
            # requests.get takes base_url and params
            base_url,
            # parameters for get request
            params = {
                # specify subreddit
                'subreddit' : subreddit,
                # specify number of posts to pull
                'size' : 100,
                # ???
                'lang' : True,
                # pull everything from current time backward
                'before' : current_time }
        )
        # take data from most recent request, store as df
        df = pd.DataFrame(res.json()['data'])
        # pull specific columns from dataframe for analysis
        df = df.loc[:, ['title',
                        'author',
                        'created_utc', 
                        'selftext',
                        'subreddit',
                        'media_only',
                        'permalink']]
        # append to empty dataframe list
        df_list.append(df)
        
        #add waittime
        time.sleep(3)
        # set current time counter back to last epoch in recently grabbed df
        current_time = df['created_utc'].min()
    # return one dataframe for all requests
    return pd.concat(df_list, axis=0)
# Adapated from Tim Book's Lesson Example

In [21]:
wall = get_posts('wallstreetbets',10,1601384439)


(1000, 7)

In [22]:
wall.shape

(1000, 7)

In [24]:
wall.tail(2)

,title,author,created_utc,selftext,subreddit,media_only,permalink
98,ms paint prediction,beholdtheflesh,1601158205,,wallstreetbets,False,/r/wallstreetbets/comments/j0eznp/ms_paint_pre...
99,Got a nice pullback on WTI Oil last 8th of Sep...,cobaltor_72,1601158017,I am beta testing a web application with nice ...,wallstreetbets,False,/r/wallstreetbets/comments/j0exmd/got_a_nice_p...


In [25]:
wall_2 = get_posts('wallstreetbets',10,1601158017)

In [31]:
wall_2.tail()

,title,author,created_utc,selftext,subreddit,media_only,permalink
95,Give me your twitter sources and I will check ...,data_diver,1600959836,"Not sure if this has been done before. But, I'...",wallstreetbets,False,/r/wallstreetbets/comments/iyywd3/give_me_your...
96,"FMCI Extension vote, IF YOU OWN SHARES VOTE NOW!",patcarnig,1600959751,FMCI -&gt; Tattoed Chef had their extension vo...,wallstreetbets,False,/r/wallstreetbets/comments/iyyvgd/fmci_extensi...
97,My gains over the last few days...,mansaodokann,1600959745,,wallstreetbets,False,/r/wallstreetbets/comments/iyyvdj/my_gains_ove...
98,Why isn't SBE going up today?,WashNo2592,1600959190,,wallstreetbets,False,/r/wallstreetbets/comments/iyypax/why_isnt_sbe...
99,If SPY doesn’t close red today imma be pissed,anotherexstnslcrisis,1600959125,,wallstreetbets,False,/r/wallstreetbets/comments/iyyok5/if_spy_doesn...


In [26]:
wall_2.shape

(1000, 7)

In [32]:
wall_3 = get_posts('wallstreetbets',15,1600959125)

In [34]:
wall_3.shape

(1500, 7)

In [52]:
wall_3.tail(1)

,title,author,created_utc,selftext,subreddit,media_only,permalink
99,Just a reminder,thatguyfrom2020,1600816260,,wallstreetbets,False,/r/wallstreetbets/comments/ixyji0/just_a_remin...


In [53]:
wall_4 = get_posts('wallstreetbets',5,1600816260)

In [54]:
wall_4.shape

(500, 7)

In [55]:
final_wall = pd.concat([wall,wall_2,wall_3,wall_4])

In [56]:
final_wall.shape

(4000, 7)

In [43]:
ml = get_posts('machinelearning',20,None)

In [45]:
ml.shape

(2000, 7)

In [46]:
ml.tail(1)

,title,author,created_utc,selftext,subreddit,media_only,permalink
99,Unsupervised Learning: Tesla stock prices (and...,Singular23,1599180384,,MachineLearning,False,/r/MachineLearning/comments/im60bx/unsupervise...


In [47]:
ml_2 = get_posts('machinelearning',20,1599180384)

In [48]:
ml_2.shape

(2000, 7)

In [49]:
final_ml = pd.concat([ml,ml_2])

In [50]:
final_ml.shape

(4000, 7)

In [57]:
final_both = pd.concat([final_wall,final_ml])

In [58]:
final_both.shape

(8000, 7)

In [59]:
pwd

'C:\\Users\\seanp\\DSIR_824\\submissions\\projects\\Project-3'

In [61]:
final_both.to_csv('scraped_sr.csv')

In [62]:
final_both.isnull().sum()

import spacy
from sklearn.pipeline import make_pipeline

In [226]:
X = final_both['title']

y = final_both['subreddit']

y.value_counts()

y = y.map({'MachineLearning':0,'wallstreetbets':1})

y.value_counts()

from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42,test_size=.4)

pipe = make_pipeline(TfidfVectorizer(),MultinomialNB())

pipe.fit(X_train,y_train)

pipe.score(X_test,y_test)

pipe.get_params().keys()

params = {
    'tfidfvectorizer__max_features': range(3000,6001,250),
    'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
#     'multinomialnb__alpha':[.01,.1,1,2]
}

gs = GridSearchCV(pipe,params)

import sklearn

sklearn.set_config(display='diagram')

gs.fit(X_train,y_train)

gs.best_params_

gs.score(X_test,y_test)

gs.best_score_

from sklearn.metrics import plot_confusion_matrix, classification_report

plot_confusion_matrix(gs,X_test,y_test)

plot_confusion_matrix(pipe,X_test,y_test)



print(classification_report(y_test,gs.predict(X_test)))

print(classification_report(y_test,pipe.predict(X_test)))



pipe_2 = make_pipeline(CountVectorizer(),MultinomialNB())

pipe_2.fit(X_train,y_train)

pipe_2.score(X_test,y_test)

print(classification_report(y_test,pipe_2.predict(X_test)))

plot_confusion_matrix(pipe,X_train,y_train)

plot_confusion_matrix(pipe,X_test,y_test)